In [ ]:
from sklearn.datasets import load_boston
import xgboost
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
#from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
target1 = pd.read_csv('/content/drive/MyDrive/AIfactory_에너지 인공지능 경진대회/기압/pressure_actual.csv')
target1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43824 entries, 0 to 43823
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   지점         43824 non-null  int64  
 1   지점명        43824 non-null  object 
 2   일시         43824 non-null  object 
 3   기온(°C)     43818 non-null  float64
 4   강수량(mm)    4537 non-null   float64
 5   풍속(m/s)    43823 non-null  float64
 6   풍향(16방위)   43823 non-null  float64
 7   습도(%)      43823 non-null  float64
 8   현지기압(hPa)  43822 non-null  float64
 9   일조(hr)     23935 non-null  float64
 10  일사(MJ/m2)  23922 non-null  float64
 11  적설(cm)     1020 non-null   float64
 12  전운량(10분위)  39842 non-null  float64
dtypes: float64(10), int64(1), object(2)
memory usage: 4.3+ MB


In [ ]:
target1 = pd.read_csv('/content/drive/MyDrive/AIfactory_에너지 인공지능 경진대회/기압/pressure_actual.csv')

In [ ]:
# 기압만 가져오기 
target = pd.read_csv('/content/drive/MyDrive/AIfactory_에너지 인공지능 경진대회/기압/pressure_actual.csv')
target = target[['일시', '현지기압(hPa)']]
target.columns =['datetime', 'pressure']

target['datetime'] = pd.to_datetime(target['datetime'])
target = target[target['datetime']>='2017-05-01'].reset_index(drop = True)

target.shape #5년치 pressure 

(36528, 2)

In [ ]:
target2 = pd.read_csv('/content/drive/MyDrive/AIfactory_에너지 인공지능 경진대회/기압/data/GJ210701-220630 기압만.csv', encoding='CP949')
target2 = target2[['일시','현지기압(hPa)']]
target2.columns = ['datetime', 'pressure']
target2['datetime'] = pd.to_datetime(target2['datetime'])

In [ ]:
target = pd.concat([target,target2], axis=0)
target.reset_index(drop = True, inplace = True)
target.sort_values(by=['datetime'], ascending= True)

,datetime,pressure
45257,2015-05-01 00:00:00,1006.3
45258,2015-05-01 01:00:00,1006.2
45259,2015-05-01 02:00:00,1005.6
45260,2015-05-01 03:00:00,1005.7
45261,2015-05-01 04:00:00,1005.5
...,...,...
45252,2022-06-30 19:00:00,1003.8
45253,2022-06-30 20:00:00,1004.5
45254,2022-06-30 21:00:00,1005.4
45255,2022-06-30 22:00:00,1006.0


In [ ]:
target = target.sort_values(by=['datetime'], ascending= True)
target.reset_index(drop=True, inplace=True)

In [ ]:
data_old = pd.read_csv('/content/drive/MyDrive/AIfactory_에너지 인공지능 경진대회/기압/pressure_forecast_old.csv') 
data_old = data_old[data_old['datetime']>='2017-05-01'] 
data_old = data_old[:36528].reset_index(drop=True)
data_old.dropna(inplace=True)

def make_datetime(data_old): 
  data_old['datetime'] = pd.to_datetime(data_old['datetime'])
  data_old['year'] = data_old['datetime'].dt.year 
  data_old['month'] = data_old['datetime'].dt.month 
  data_old['day'] = data_old['datetime'].dt.day
  data_old['hour'] = data_old['datetime'].dt.hour
  return data_old

data_old = make_datetime(data_old)

In [ ]:
data_new = pd.read_csv('/content/drive/MyDrive/AIfactory_에너지 인공지능 경진대회/20220806/wind_train_14.csv')
data_new['datetime'] = pd.to_datetime(data_new['datetime'])
data_new = data_new[data_new['datetime']>='2021-07-01'][['datetime', 'year', 'month', 'day', 'hour', 'temperature', 'precipitationform', 'precipitationprob',
       'humidity', 'windspeed', 'winddirection', 'cloud', 'precipitation', 'snow']]
data_test = pd.read_csv('/content/drive/MyDrive/AIfactory_에너지 인공지능 경진대회/20220806/wind_test_14.csv') 
data_test = data_test[['datetime', 'year', 'month', 'day', 'hour', 'temperature', 'precipitationform', 'precipitationprob',
       'humidity', 'windspeed', 'winddirection', 'cloud', 'precipitation', 'snow']]
data_new.head()       

,datetime,year,month,day,hour,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,snow
7344,2021-07-01 00:00:00,2021,7,1,0,20.000000,0.0,0.0,90.0,1.1,90.0,1.0,0.0,0.0
7345,2021-07-01 01:00:00,2021,7,1,1,19.666667,0.0,0.0,90.0,1.2,90.0,1.0,0.0,0.0
7346,2021-07-01 02:00:00,2021,7,1,2,19.333333,0.0,0.0,90.0,1.1,85.0,1.0,0.0,0.0
7347,2021-07-01 03:00:00,2021,7,1,3,19.000000,0.0,0.0,90.0,1.1,80.0,1.0,0.0,0.0
7348,2021-07-01 04:00:00,2021,7,1,4,18.666667,0.0,0.0,90.0,1.1,80.0,1.0,0.0,0.0


In [ ]:
total_df = pd.concat([data_old,data_new,data_test], axis=0)
total_df = total_df.drop(['snow'], axis=1)
total_df.reset_index(drop=True, inplace=True)
total_df.shape

(45288, 13)

In [ ]:
total_df.shape, target.shape

((48216, 13), (48185, 2))

In [ ]:
target['datetime'] = target['datetime'].astype(str)
total_df['datetime'] = total_df['datetime'].astype(str)

In [ ]:
df = pd.merge(total_df, target, on='datetime', how='left') 

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,year,month,day,hour,pressure
0,2015-05-01 00:00:00,16.500000,0.0,20.000000,85.000000,0.900000,296.000000,3.000000,0.0,2015,5,1,0,1006.3
1,2015-05-01 01:00:00,16.466667,0.0,16.666667,86.666667,0.900000,288.333333,2.666667,0.0,2015,5,1,1,1006.2
2,2015-05-01 02:00:00,16.433333,0.0,13.333333,88.333333,0.900000,280.666667,2.333333,0.0,2015,5,1,2,1005.6
3,2015-05-01 03:00:00,16.400000,0.0,10.000000,90.000000,0.900000,273.000000,2.000000,0.0,2015,5,1,3,1005.7
4,2015-05-01 04:00:00,16.100000,0.0,10.000000,90.333333,0.933333,182.666667,2.000000,0.0,2015,5,1,4,1005.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48211,2022-06-30 19:00:00,27.000000,0.0,20.000000,80.000000,1.500000,164.000000,3.000000,0.0,2022,6,30,19,1003.8
48212,2022-06-30 20:00:00,26.000000,0.0,20.000000,85.000000,1.700000,149.000000,3.000000,0.0,2022,6,30,20,1004.5
48213,2022-06-30 21:00:00,25.000000,0.0,20.000000,90.000000,1.900000,148.000000,3.000000,0.0,2022,6,30,21,1005.4
48214,2022-06-30 22:00:00,24.000000,0.0,20.000000,90.000000,1.900000,152.000000,3.000000,0.0,2022,6,30,22,1006.0


In [ ]:
df.isna().sum()

datetime              0
temperature           0
precipitationform     0
precipitationprob     0
humidity              0
windspeed             0
winddirection         0
cloud                 0
precipitation         0
year                  0
month                 0
day                   0
hour                  0
pressure             33
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

(48183, 14)

In [ ]:
# 5,6 월 2017 train 데이타셋 만들기    

In [ ]:
df2 = df[(df['datetime'] <= '2022-05-01') & (df['datetime'] >= '2017-05-01')]
train3 = df2[(df2['month'] == 5) | (df2['month'] == 6)] 
train3.reset_index(drop=True, inplace=True)
train3.to_csv('/content/drive/MyDrive/AIfactory_에너지 인공지능 경진대회/data_for기압모델/train_2017_56_fcstpres.csv', index = False)

In [ ]:
df.to_csv('/content/drive/MyDrive/AIfactory_에너지 인공지능 경진대회/기압/new_code_관측_예보/df_20152022_예보기압.csv', index=False)